In [1]:
import os
os.chdir('../')
os.getcwd()

'e:\\github_clone\\Trader-RL'

In [2]:
import gymnasium as gym
from src.agent import RLSeq2Seq, Encoder, Decoder
from src.trainer import Seq2SeqTrainer
from src.utility.logger import logger
from src.utility.data_prep import build_trading_features
from src.utility.config import config
import gymnasium as gym
import numpy as np
import gym_trading_env
import torch

In [3]:
url = "https://raw.githubusercontent.com/ClementPerroud/Gym-Trading-Env/main/examples/data/BTC_USD-Hourly.csv"
df = build_trading_features(source=url)

In [4]:
df.head()

,unix,symbol,open,high,low,close,volume,Volume USD,feature_close,feature_open,feature_high,feature_low,feature_volume
date,,,,,,,,,,,,,
2018-05-22 05:00:00,1526965200,BTC/USD,8285.29,8314.32,8261.52,8280.00,1711668.56,206.40,-0.000638,1.000639,1.004145,0.997768,0.120688
2018-05-22 06:00:00,1526968800,BTC/USD,8280.00,8355.97,8280.00,8317.02,2975233.66,357.53,0.004471,0.995549,1.004683,0.995549,0.209057
2018-05-22 07:00:00,1526972400,BTC/USD,8317.02,8333.49,8297.00,8320.16,893167.95,107.43,0.000378,0.999623,1.001602,0.997216,0.062817
2018-05-22 08:00:00,1526976000,BTC/USD,8320.16,8325.60,8290.00,8295.90,968758.33,116.54,-0.002916,1.002924,1.003580,0.999289,0.068144
2018-05-22 09:00:00,1526979600,BTC/USD,8295.90,8296.01,8220.00,8245.47,3954060.34,479.07,-0.006079,1.006116,1.006129,0.996911,0.280125


In [5]:
env = gym.make("TradingEnv",
        name= "BTCUSD",
        df = df, # Your dataset with your custom features
        positions = [ -1, 0, 1], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
        trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
        borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
    )

In [6]:
config['input_dim'] = env.observation_space.shape[0]
config['action_dim'] = env.action_space.n

In [7]:
encoder = Encoder(config)
decoder = Decoder(config)
agent   = RLSeq2Seq(encoder, decoder, config)

In [8]:
def one_hot(a, n):
    v = np.zeros(n, dtype=np.float32); v[int(a)] = 1.0; return v

def build_prev_context(state_vec, prev_action, action_dim):
    # your Decoder.context_embed expects (input_dim + action_dim)
    oh = np.zeros(action_dim, dtype=np.float32) if prev_action is None else one_hot(prev_action, action_dim)
    return np.concatenate([state_vec, oh], axis=0)

In [9]:
def rollout_and_update(env, agent:RLSeq2Seq, max_steps=1000):
    terminated = truncated = False
    obs, info = env.reset()
    prev_action = None
    ep_ret = 0.0

    while not (terminated or truncated) and max_steps > 0:
        state_vec = np.array(obs, dtype=np.float32).reshape(-1)
        action_dim = env.action_space.n
        prev_ctx = build_prev_context(state_vec, prev_action, action_dim)

        # uses your (now fixed) select_action
        action, _ = agent.select_action(state_vec, prev_ctx)

        obs_next, reward, terminated, truncated, info = env.step(action)
        ep_ret += float(reward)

        # buffer fields your update() needs
        agent.buffer.rewards.append(float(reward))
        agent.buffer.is_terminals.append(terminated or truncated)
        agent.buffer.prev_contexts.append(torch.tensor(prev_ctx, dtype=torch.float32, device=agent.device))
        agent.buffer.next_state.append(torch.tensor(np.array(obs_next, dtype=np.float32).reshape(-1), device=agent.device))

        prev_action = int(action)
        obs = obs_next
        max_steps -= 1

    return ep_ret


In [11]:
rollout_and_update(env, agent, 1000)

0.1560243277631689